In [1]:
# ! pip install transformers
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer

import torchsummary as summary
from tqdm import tqdm

from torch import nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os
import base64
import gc
import glob, os
import random

from torch.optim import Adam
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import json
from general_functions import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import data
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and dowload the files
aws = awsOps(aws_creds_json)
df = aws.get_df(bucket=bucket, file="data_rule_labels_v1.csv")
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content']]
df = df.reset_index()
del df['index']
df=df[:40]
df

,news_content
0,Berenberg Bank analysts have provided a buy ra...
1,"The article states that Berenberg, a German in..."
2,"In their analysis on October 30, 2023, experts..."
3,"Berenberg Bank has issued a \""buy\"" recommenda..."
4,The private bank Berenberg has upgraded its ra...
...,...
14496,INTERVIEW Interview with Les Échos Interview w...
14497,UBS's latest Investor Watch report reveals tha...
14498,SNB erwartet für 2021 Jahresgewinn von rund 26...
14499,0:00 News A cryptocurrency exchange in Hong Ko...


In [3]:
def dynamic_whole_word_masking(df=df, column_name='news_content', mask_probability=0.15):
    # Initialize BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Function to mask tokens of a sentence
    def mask_sentence(sentence):
        tokens = tokenizer.tokenize(sentence)
        new_tokens = []
        mask_indices = random.sample(range(len(tokens)), int(len(tokens) * mask_probability))

        for i, token in enumerate(tokens):
            # Check if the token is part of a word to be masked
            if i in mask_indices or (i > 0 and tokens[i-1] in mask_indices and token.startswith('##')):
                new_tokens.append('[MASK]')
            else:
                new_tokens.append(token)

        return tokenizer.convert_tokens_to_string(new_tokens)

    # Apply masking to each row in the DataFrame
    df[column_name] = df[column_name].apply(mask_sentence)

    return df


In [5]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [6]:
df_train, df_val = train_test_split(df[['news_content']], test_size=0.25)

In [7]:
df_test=0

In [8]:
from transformers import AutoTokenizer, BertForMaskedLM
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model=model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)

df_val=df_val.reset_index()
df_val.drop('index',axis=1,inplace=True)
df_val

,news_content
0,Dass die Commerzbank (WKN: CBK100) ihre Aktion...
1,"Le procès de Christian Olearius, copropriétair..."
2,Berenberg Bank's target price would indicate a...
3,The former Chairman of a Swiss major bank serv...
4,"La reprise économique en Allemagne, qui avait ..."
...,...
3621,“Watching Grant moving forward in his dreams b...
3622,Nach mehr als 20 Jahren im Dienst der UBS wech...
3623,The article discusses the sluggish development...
3624,French prosecutors have launched an appeal aga...


In [10]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

204

In [11]:
# freeze first 8 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [13]:
# change datatypes of input data
df_train['news_content']=df_train['news_content'].astype(str)

df_val['news_content']=df_val['news_content'].astype(str)
df_train['news_content'].iloc[0]

'Die Commerzbank und der Energieanbieter RWE haben eine Absichtserklärung unterzeichnet wonach sie mittelständischen Unternehmen langfristige Lieferverträge für den Bezug von Ökostrom aus OffshoreWindkraftanlagen sichern wollen. Über einen grünen Mittelstandsfonds“ erhalten die Firmen Zugang zu sogenannten Power Purchase Agreements (PPAs) das sind langfristige StromabnahmeVereinbarungen an die bislang nur Großverbraucher aus der Industrie gelangen können. Für CommerzbankKunden tut sich...'

In [14]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [16]:
torch.cuda.empty_cache()
gc.collect()

0

In [17]:
import sys
sys.path.append("..")
# Add the parent directory to the Python path
from reghub_pack.models import BERT_RegHub_Similarity

In [18]:
model = BERT_RegHub_Similarity()
model.load_mlm()
model.model_training(train_data=train_data, val_data=val_data)

Epoch 1/250: 100%|████████████████████████████████████████████████████████| 544/544 [56:35<00:00,  6.24s/item, Epoch=1]


Epoch 2/250: 100%|████████████████████████████████████████████████████████| 544/544 [56:32<00:00,  6.24s/item, Epoch=2]


Epoch 3/250: 100%|████████████████████████████████████████████████████████| 544/544 [56:32<00:00,  6.24s/item, Epoch=3]


Epoch 4/250: 100%|████████████████████████████████████████████████████████| 544/544 [56:32<00:00,  6.24s/item, Epoch=4]


Epoch 5/250: 100%|████████████████████████████████████████████████████████| 544/544 [56:32<00:00,  6.24s/item, Epoch=5]


...

Epoch 21/250: 100%|██████████████████████████████████████████████████████| 544/544 [56:24<00:00,  6.22s/item, Epoch=21]


Epoch 22/250: 100%|██████████████████████████████████████████████████████| 544/544 [56:23<00:00,  6.22s/item, Epoch=22]


Epoch 23/250: 100%|██████████████████████████████████████████████████████| 544/544 [56:21<00:00,  6.22s/item, Epoch=23]


Epoch 24/250:   0%|                                                                          | 0/544 [00:05<?, ?item/s]


KeyboardInterrupt: 

In [19]:
model.delete_MLM()
model.weight_transfer()
model.save_model(save_aws=False)